In [ ]:
# Importing Libraries required.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.utils import resample

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [ ]:
# Load the dataset from the CSV file
study_data= pd.read_csv("mental-heath-in-tech-2016_20161114.csv")

In [ ]:
for col in study_data.columns:
    print(col)

In [ ]:
# Make a copy of the original colums to compare later
orig_columns = study_data.columns

# Rename the columns for better readability
study_data.columns = [
    "self_employed",
    "num_employees",
    "employer_tech_company",
    "role_related_to_tech",
    "mental_health_benefits",
    "know_mental_health_options",
    "employer_discussed_mh",
    "employer_mh_resources",
    "anonymity_protected",
    "mh_leave_comfort",
    "mh_discussion_negative",
    "ph_discussion_negative",
    "mh_comfort_coworkers",
    "mh_comfort_supervisor",
    "employer_mh_priority",
    "negative_consequences_observed",
    "medical_coverage_mh",
    "know_local_mh_resources",
    "reveal_mh_clients",
    "reveal_mh_clients_negative",
    "reveal_mh_coworkers",
    "reveal_mh_coworkers_negative",
    "productivity_affected",
    "work_time_affected_pct",
    "previous_employers",
    "prev_employers_mh_benefits",
    "prev_employers_mh_options",
    "prev_employers_discussed_mh",
    "prev_employers_mh_resources",
    "prev_employers_anonymity",
    "prev_employers_mh_negative",
    "prev_employers_ph_negative",
    "mh_comfort_prev_coworkers",
    "mh_comfort_prev_supervisor",
    "prev_employers_mh_priority",
    "prev_employers_negative_obs",
    "physical_health_in_interview",
    "physical_health_in_interview_reason",
    "mental_health_in_interview",
    "mental_health_in_interview_reason",
    "mh_hurt_career",
    "mh_viewed_negatively",
    "mh_share_friends_family",
    "unsupportive_response",
    "observed_mh_discussion_effect",
    "family_history_mh",
    "past_mh_disorder",
    "current_mh_disorder",
    "current_mh_condition",
    "maybe_mh_condition",
    "diagnosed_mh_condition",
    "diagnosed_mh_condition_details",
    "sought_mh_treatment",
    "mh_treatment_effective",
    "mh_treatment_ineffective",
    "age",
    "gender",
    "country_residence",
    "us_state_residence",
    "country_work",
    "us_state_work",
    "work_position",
    "work_remote"
]

print(orig_columns)

In [ ]:
# Compare the original columns with the new columns
for i in range(len(orig_columns)):
    print(orig_columns[i], " -> ", study_data.columns[i])

# Get a brief overview of the data

In [ ]:
# Display main statistics of each column
describe = study_data.describe(include='all').T.to_string()
print(describe)

In [ ]:
# Display all unique values of each column
for col in study_data.columns:
    print("\n",study_data[col].value_counts(), "\n")

# Data cleaning

## Remove answers with no informations

In [ ]:
# Removing uncertain answers  
study_data = study_data.replace("Not applicable to me",np.nan)
study_data = study_data.replace("I don't know",np.nan)
study_data = study_data.replace("I'm not sure",np.nan)
study_data = study_data.replace("N/A (not currently aware)",np.nan)
study_data = study_data.replace("Not eligible for coverage / N/A",np.nan)
study_data = study_data.replace("Not applicable to me (I do not have a mental illness)", np.nan)

## Unify country names

In [ ]:
# Define a mapping for replacement
country_mapping = {
    'United States of America': 'USA',
    'United Kingdom': 'UK'
}

# Replace values in both 'country_live' and 'country_work' columns
study_data[['country_residence', 'country_work']] = study_data[['country_residence', 'country_work']].replace(country_mapping)

## Drop redundant and unecessary features

In [ ]:
# Check where county_work is different from country_residence
i = 0
for index, row in study_data.iterrows():
    if row['country_residence'] != row['country_work']:
        print(row['country_residence'], " -> ", row['country_work'])
        i += 1

print("\nTotal number of people having different work than residence countries: ", i/index*100, "%")


#### Drop country_residence, us_state_residence, us_state_work because the information is not relevant for the topic

In [ ]:
study_data.drop(columns=["country_residence"], inplace=True)    # Drop the country_residence column
study_data.drop(columns=["us_state_residence"], inplace=True)   # Drop the us_state_residence column
study_data.drop(columns=["us_state_work"], inplace=True)        # Drop the us_state_work column

#### Find coulums with contextual answers

In [ ]:
# find columns with strings greater than 10 characters
for col in study_data.columns:
    if study_data[col].dtype == 'object':
        if study_data[col].str.len().max() > 10:
            print("\n",study_data[col].value_counts(), "\n")

In [ ]:
# Drop the columns with contextual data
study_data.drop(columns=["physical_health_in_interview_reason"], inplace=True)
study_data.drop(columns=["mental_health_in_interview_reason"], inplace=True)


#### Replace 

## Replace age outliers

In [ ]:
print(study_data['age'].unique())       # Display unique age values

In [ ]:
age_copy = study_data['age'].copy()                     # Create a copy for later comparison
study_data['age'] = study_data['age'].apply(
    lambda x: np.nan if x < 15 or x > 75 else x         # Replace age outliers with NaN
    )
age_checksum = (study_data['age'] != age_copy).sum()    # Check how many rows were changed

print(f"Replaced age outliers with NaN for {age_checksum} rows.")

## Fill missing age values

In [ ]:
age_copy_2 = study_data['age'].copy()                       # Create a copy for later comparison
age_median = study_data['age'].median()                     # Calculate the median age
study_data.fillna({'age': age_median}, inplace=True)        # Fill missing age values with the median
age_checksum_2 = (age_copy_2 != study_data['age']).sum()    # Check how many rows were changed

print(f"Filled missing age values with the median: {age_median}  for {age_checksum_2} rows.")

## Reduce age to categories

In [ ]:
study_data['age'] = pd.cut(
    study_data['age'], 
    bins=[18, 30, 40, 50, 60, float('inf')],  # Extend bins to include all ages above 60
    labels=['18-30', '30-40', '40-50', '50-60', '60+'], 
    right=False
)
print(study_data['age'].value_counts())

## Replace company size 

In [ ]:
study_data['num_employees'] = study_data['num_employees'].replace('More than 1000', '1000+')

## Drop rows with too many missing values


In [ ]:
original_count = study_data.shape[0]                                            # Get the original row count
study_data = study_data.dropna(thresh=study_data.shape[1] * 0.65)               # Drop rows with over 35% missing values
drop_lines_percent = (1 - (study_data.shape[0] / original_count)) *100          # Calculate the percentage of dropped rows

print(f"Dropped {drop_lines_percent}% of rows due to missing values.")

## Drop columns with to many missing values

#### Get an overview of the nan percentage in the columns

In [ ]:
# Calculate NaN percentages
nan_percentage = (study_data.isna().sum() / len(study_data)) * 100
# Sort by percentage in descending order
nan_percentage_sorted = nan_percentage.sort_values(ascending=False)

# Display the sorted percentages
print(nan_percentage_sorted.to_string())

#### Drop columns with more than 50% of missing values

In [ ]:
for col_name, percentage in nan_percentage_sorted.items():         # Iterate over the sorted percentages 
    if percentage > 50:                                            # If the percentage is over 30%
        study_data.drop(col_name, axis=1, inplace=True)            # Drop the column from study_data

#### Check for missing values percentage again

In [ ]:
nan_percentage = (study_data.isna().sum() / len(study_data)) * 100  # Recalculate NaN percentages
nan_percentage_sorted = nan_percentage.sort_values(ascending=False) # Sort by percentage in descending order

print(nan_percentage_sorted.to_string())                            # Display the sorted percentages

## Replace genders

In [ ]:
# Define gender categories with a dictionary for efficient replacement
gender_map = {
    **{key: "male" for key in ['Male', 'male', 'Male ', 'M', 'm', 'man', 'Cis male', 'Male.', 
                               'male 9:1 female, roughly', 'Male (cis)', 'Man', 'Sex is male',
                               'cis male', 'Malr', 'Dude', 
                               "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
                               'mail', 'M|', 'Male/genderqueer', 'male ', 'Cis Male', 
                               'Male (trans, FtM)', 'cisdude', 'cis man', 'MALE']},
                               
    **{key: "female" for key in ['Female', 'female', 'I identify as female.', 'female ', 
                                 'Female assigned at birth ',
                                 'F', 'Woman', 'fm', 'f', 'Cis female ', 'Transitioned, M2F',
                                 'Genderfluid (born female)', 'Female or Multi-Gender Femme', 
                                 'Female ', 'woman', 'female/woman', 'Cisgender Female', 'fem', 
                                 'Female (props for making this a freeform field, though)', 
                                 ' Female', 'Cis-woman', 'female-bodied; no feelings about gender',
                                 'AFAB']}
}

# Store initial gender value counts
gender_before = study_data["gender"].value_counts()

# Replace genders using the map and set all non-male/female values to 1
study_data["gender"] = study_data["gender"].replace(gender_map).apply(
    lambda x: "queer" if x not in ["male", "female"] else x
    )

# Display results
print("Before:\n", gender_before, "\n\n\nAfter:\n", study_data["gender"].value_counts())

## Reducing countries of work

In [ ]:
# Get the unique values of the 'country_work' column
print(study_data['country_work'].value_counts())

In [ ]:
# Map countries to continents
# Define the continent mappings
continent_mapping = {
    **{country: "north_america" for country in ["United States of America", "Canada", "Mexico"]},
    **{country: "europe" for country in [
        "United Kingdom", "Netherlands", "Germany", "Sweden", "France",
        "Ireland", "Switzerland", "Bulgaria", "Finland", "Denmark",
        "Russia", "Spain", "Norway", "Austria", "Bosnia and Herzegovina",
        "Italy", "Poland", "Belgium", "Czech Republic"]},
}

# Replace the values in 'country_work' column with the mapped values or "Other"
study_data['country_work'] = study_data['country_work'].apply(lambda x: continent_mapping.get(x, "Other"))

# Display the unique values of the 'country_work' column
print(study_data['country_work'].value_counts())

## Reducing work position


In [ ]:
# Get the unique values of the 'work_position' column
print(study_data['work_position'].value_counts().to_string())

In [ ]:
# Optimized category mapping with direct keyword-to-category association
keyword_to_category = {
    "supervisor": "Management",
    "team lead": "Management",
    "leadership": "Management",
    "executive": "Management",
    "back": "Development",
    "developer": "Development",
    "front": "Development",
    "devops": "Development",
    "sysadmin": "Development",
    "dev": "Development",
    "support": "Support",
    "helpdesk": "Support",
    "customer": "Support",
    "design": "Design",
    "ui": "Design",
    "ux": "Design",
    "creative": "Design",
    "sales": "Sales",
    "marketing": "Sales",
    "business": "Sales",
    "evangelist": "Advocacy",
    "advocate": "Advocacy",
    "hr": "Human Resources",
    "human resources": "Human Resources",
    "recruitment": "Human Resources",
}

# Function to classify roles
def categorize_role_optimized(role):
    role = role.lower()  # Standardize to lowercase
    for keyword, category in keyword_to_category.items():
        if keyword in role:
            return category
    return "Other"  # Default for unmatched roles

# Convert the entire column to lowercase for efficiency
study_data['work_position'] = study_data['work_position'].str.lower()

# Apply the optimized categorization function
study_data['work_position'] = study_data['work_position'].apply(categorize_role_optimized)

# Display the unique values and their counts
print(study_data['work_position'].value_counts().to_string())

## Display all unique values for each column

In [ ]:
for col in study_data.columns:
    print("\n\n", study_data[col].value_counts().to_string(), "\n\n")

Encode the  diagnosed_mh_condition_details column

In [ ]:
# Create a set to store the individual values
unique_values = set()

# Iterate over the rows and save the unique strings split at '|'
for value in study_data['diagnosed_mh_condition_details']:
    if pd.notna(value):
        unique_values.update(value.split('|'))

# Display the unique values
print("\n".join( unique_values))

In [ ]:
# Iterate over the rows and split at '|'
study_data['diagnosed_mh_condition_details'] = study_data['diagnosed_mh_condition_details'].apply(
    lambda x: x.split('|') if pd.notna(x) else x
)

In [ ]:
# Define the mapping of disorders to categories
disorder_category_mapping = {
    # Mood Disorders
    "Depression": "Mood Disorders",
    "Seasonal Affective Disorder": "Mood Disorders",
    "Mood Disorder (Depression, Bipolar Disorder, etc)": "Mood Disorders",
    "Suicidal Ideation": "Mood Disorders",
    "Intimate Disorder": "Mood Disorders",

    # Anxiety and Stress-Related Disorders
    "Anxiety Disorder (Generalized, Social, Phobia, etc)": "Anxiety and Stress-Related Disorders",
    "Post-traumatic Stress Disorder": "Anxiety and Stress-Related Disorders",
    "Stress Response Syndromes": "Anxiety and Stress-Related Disorders",
    "Obsessive-Compulsive Disorder": "Anxiety and Stress-Related Disorders",
    "Gender Identity Disorder": "Anxiety and Stress-Related Disorders",
     "posttraumatic stress disourder": "Anxiety and Stress-Related Disorders",

    # Neurodevelopmental Disorders
    "Attention Deficit Hyperactivity Disorder": "Neurodevelopmental Disorders",
    "ADD (w/o Hyperactivity)": "Neurodevelopmental Disorders",
    "Attention Deficit Disorder": "Neurodevelopmental Disorders",
    "MCD (when it was diagnosed, the ultra-mega 'disorder' ADHD didn't exist yet)": "Neurodevelopmental Disorders",
    "Autism Spectrum Disorder": "Neurodevelopmental Disorders",
    "Autism": "Neurodevelopmental Disorders",
    "Autism - while not a 'mental illness', still greatly affects how I handle anxiety": "Neurodevelopmental Disorders",
    "Asperger Syndrome": "Neurodevelopmental Disorders",
    "Aspergers": "Neurodevelopmental Disorders",

    # Personality Disorders
    "Personality Disorder (Borderline, Antisocial, Paranoid, etc)": "Personality Disorders",
    "Gender Dysphoria": "Personality Disorders",
    "Eating Disorder (Anorexia, Bulimia, etc)": "Personality Disorders",

    # Psychotic Disorders
    "Psychotic Disorder (Schizophrenia, Schizoaffective, etc)": "Psychotic Disorders",
    "Dissociative Disorder": "Dissociative Disorders",

    # Substance-Related and Addictive Disorders
    "Substance Use Disorder": "Substance-Related and Addictive Disorders",
    "Addictive Disorder": "Substance-Related and Addictive Disorders",
}

def map_disorders_to_categories(disorder_list):
    if isinstance(disorder_list, list):  # Check if input is a list
        # Map the list
        mapped_categories = []
        for item in disorder_list:
            mapped_categories.append(disorder_category_mapping.get(item, "Other"))
        return mapped_categories
    else:
        return disorder_list  # Return as-is if not a list
    
# Apply the mapping to the DataFrame
study_data['diagnosed_mh_condition_details'] = study_data['diagnosed_mh_condition_details'].apply(map_disorders_to_categories)

# Take the mode of each row in the 'diagnosed_mh_condition_details' column
study_data['diagnosed_mh_condition_details'] = study_data['diagnosed_mh_condition_details'].apply(
    lambda x: max(set(x), key=x.count) if isinstance(x, list) else x
)
# Drop Other and Psychotic Disorders
study_data['diagnosed_mh_condition_details'] = study_data['diagnosed_mh_condition_details'].replace('Other', np.nan)
study_data['diagnosed_mh_condition_details'] = study_data['diagnosed_mh_condition_details'].replace('Psychotic Disorders', np.nan)

In [ ]:
print(study_data['diagnosed_mh_condition_details'].value_counts())

# Imoute missing values

#### Get an overview of the missing data

In [ ]:
# Display the missing value counts for each column and the most frequent value
for col in study_data.columns:
    print(f"\n{col}:")
    print('Number of missing values: ', study_data[col].isna().sum())
    print('Most frequent value:      ', study_data[col].mode().values[0])

#### Use resample to impute the missing data based on the proportion of the existing data to preserve variance

In [ ]:
# Loop through the columns and replace missing values with the most frequent value
for col in study_data.columns:
  
    # Replace missing values statistical most likely value
    missing = study_data[col].isnull()
    probs = study_data[col].value_counts(normalize=True)
    study_data.loc[missing, col] = np.random.choice(probs.index, size=len(study_data[missing]), p=probs.values)

#### Check if inputing was successful

In [ ]:
# Create a check variable
check = True

# Check if there are any missing values left
for col in study_data.columns:
    if study_data[col].isna().sum() > 0:
        print(col)
    else:
        check = False

# If there are no missing values left, display a message
if not check:
    print("No missing values left.")

#### Final data shape

In [ ]:
study_data.shape

## Save the final data set to .csv

In [ ]:
# Save to a new CSV file
study_data.to_csv("prep_data.csv", index=False)

# Encoding

In [67]:
# Select columns based on types
numerical_cols = [
    'employer_tech_company', 
    'self_employed', 
    'sought_mh_treatment', 
    'previous_employers'
]

# All columns with defined ordinal mapping
ordinal_cols = [
    'num_employees',
    'mental_health_benefits',
    'know_mental_health_options',
    'employer_discussed_mh',
    'employer_mh_resources',
    'mh_leave_comfort',
    'mh_discussion_negative',
    'ph_discussion_negative',
    'mh_comfort_coworkers',
    'mh_comfort_supervisor',
    'employer_mh_priority',
    'negative_consequences_observed',
    'prev_employers_mh_benefits',
    'prev_employers_mh_options',
    'prev_employers_discussed_mh',
    'prev_employers_mh_resources',
    'prev_employers_mh_negative',
    'prev_employers_ph_negative',
    'mh_comfort_prev_coworkers',
    'mh_comfort_prev_supervisor',
    'prev_employers_mh_priority',
    'prev_employers_negative_obs',
    'physical_health_in_interview',
    'mental_health_in_interview',
    'mh_hurt_career',
    'mh_viewed_negatively',
    'mh_share_friends_family',
    'unsupportive_response',
    'observed_mh_discussion_effect',
    'family_history_mh',
    'past_mh_disorder',
    'current_mh_disorder',
    'diagnosed_mh_condition',
    'mh_treatment_effective',
    'mh_treatment_ineffective',
    'age',
    'gender',
    'work_remote'
]

# Remaining categorical columns (excluding ordinal columns)
categorical_cols = [
    'country_work', 
    'work_position',
    'diagnosed_mh_condition_details'
]


ordinal_mapping = {
    'num_employees': ["1-5", "6-25", "26-100", "100-500", "500-1000", "1000+"],
    'mental_health_benefits': ['No', 'Yes'],
    'know_mental_health_options': ['No', 'I am not sure', 'Yes'],
    'employer_discussed_mh': ['No', 'Maybe', 'Yes'],
    'employer_mh_resources': ['No', 'Some', 'Yes'],
    'mh_leave_comfort': ['Very difficult', 'Somewhat difficult',  'Neither easy nor difficult', 'Somewhat easy', 'Very easy'],
    'mh_discussion_negative': ['No', 'Maybe', 'Yes'],
    'ph_discussion_negative': ['No', 'Maybe', 'Yes'],
    'mh_comfort_coworkers': ['No', 'Maybe', 'Yes'],
    'mh_comfort_supervisor': ['No', 'Maybe', 'Yes'],
    'employer_mh_priority': ['No', 'Somewhat', 'Yes'],
    'negative_consequences_observed': ['No', 'Maybe', 'Yes'],
    'prev_employers_mh_benefits': ['No, none did', 'Some did', 'Yes, they all did'],
    'prev_employers_mh_options': ['No, I only became aware later', 'I was aware of some', 'Yes, I was aware of all of them'],
    'prev_employers_discussed_mh': ['None did', 'Some did', 'Yes, they all did'],
    'prev_employers_mh_resources': ['None did', 'Some did', 'Yes, they all did'],
    'prev_employers_mh_negative': ['None of them', 'Some of them', 'Yes, all of them'],
    'prev_employers_ph_negative': ['None of them', 'Some of them', 'Yes, all of them'],
    'mh_comfort_prev_coworkers': ['No, at none of my previous employers', 'Some of my previous employers', 'Yes, at all of my previous employers'],
    'mh_comfort_prev_supervisor': ['No, at none of my previous employers', 'Some of my previous employers', 'Yes, at all of my previous employers'],
    'prev_employers_mh_priority': ['None did', 'Some did', 'Yes, they all did'],
    'prev_employers_negative_obs': ['None of them', 'Some of them', 'Yes, all of them'],
    'physical_health_in_interview': ['No', 'Maybe', 'Yes'],
    'mental_health_in_interview': ['No', 'Maybe', 'Yes'],
    'mh_hurt_career': ['No, it has not', 'No, I don\'t think it would', 'Maybe', 'Yes, I think it would', 'Yes, it has'],
    'mh_viewed_negatively': ['No, they do not','No, I don\'t think they would', 'Maybe','Yes, I think they would', 'Yes, they do'],
    'mh_share_friends_family': ['Not open at all', 'Somewhat not open','Neutral', 'Somewhat open', 'Very open'],
    'unsupportive_response': ['No', 'Maybe', 'Maybe/Not sure','Yes', 'Yes, I observed', 'Yes, I experienced'],
    'observed_mh_discussion_effect': ['No', 'Maybe', 'Yes'],
    'family_history_mh': ['No', 'Yes'],
    'past_mh_disorder': ['No', 'Maybe', 'Yes'],
    'current_mh_disorder': ['No', 'Maybe', 'Yes'],
    'diagnosed_mh_condition': ['No', 'Yes'],
    'mh_treatment_effective': ['Never', 'Rarely', 'Sometimes', 'Often'],
    'mh_treatment_ineffective': ['Never', 'Rarely', 'Sometimes', 'Often'],
    'age': ['18-30', '30-40', '40-50', '50-60', '60+'],
    'gender': ['male', 'female', 'queer'],
    'work_remote': ['Never', 'Sometimes', 'Always']
}



# Check if ordinal mapping is correct
for col in study_data.columns:
    for val in study_data[col].unique():
        if col in ordinal_mapping and val not in ordinal_mapping[col]:
            print(col, '-->', val)


# Check if all ordinal columns are in the ordinal mapping
for col in ordinal_cols:
    if col not in ordinal_mapping:
        print(col)



check_list = numerical_cols + ordinal_cols + categorical_cols
for col in study_data.columns:
    if col not in check_list:
        print(col)
